In [54]:
%load_ext autoreload
%autoreload 2
%cd C:\MAD4AG
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
C:\MAD4AG


In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm



import warnings

warnings.filterwarnings('ignore')

In [56]:
df_MAD = pd.read_parquet(f'./dbs/intermediate/indi_weights.parquet')


In [57]:
df_survey_twins = pd.read_pickle(f'./dbs/intermediate/df_survey_twins.pkl')


# add region information

In [58]:
lan= pd.read_csv(f'./dbs/lan.csv', delimiter=';', dtype={'code':'string'})
lan.rename(columns={'code':'county', 'landsdelar':'region'},inplace=True)

df_MAD = pd.merge(df_MAD, lan[['county','region']], on='county')
df_survey_twins = pd.merge(df_survey_twins, lan[['county','region']], on='county')


# calculate median distance

In [59]:
avg_dist_threshold = df_survey_twins.avg_dist.median()
com_dist_threshold = df_survey_twins.com_distance.median()

#df_survey_twins.avg_dist.quantile(0.5)


# create stratum

In [60]:
df_MAD['region'] = df_MAD['region'].replace({'Svealand':'0','Göteland':'1', 'Norrland':'2'})

df_MAD['urban_density'] = df_MAD['urban_density'].replace({'A': '0', 'B': '1', 'C': '1'})

df_MAD['avg_dist_cat'] = '0'
df_MAD['avg_dist_cat'][df_MAD.avg_dist > avg_dist_threshold] = '1'

df_MAD['com_dist_cat'] = '0'
df_MAD['com_dist_cat'][df_MAD.com_dist > com_dist_threshold] = '1'

In [61]:

df_MAD['stratum'] = df_MAD['region'].astype(str) + df_MAD['urban_density'].astype(str) +  df_MAD['commute'].astype(str)+  df_MAD['avg_dist_cat'].astype(str)

In [62]:

df_MAD['stratum'][(df_MAD.commute == 1)] = df_MAD['stratum'][(df_MAD.commute == 1)] + df_MAD['com_dist_cat'][(df_MAD.commute == 1)].astype(str)

df_MAD['stratum'].value_counts()

0101     48915
0100     37723
1101     34118
1100     33219
2100     11705
11100    11666
01100    10132
01111     9421
11111     8855
2101      7158
1001      6322
0001      4944
01110     3891
21100     3872
11110     3556
1000      3215
10111     2670
2001      2399
0000      1899
00111     1636
21111     1593
11101     1560
01101     1448
2000      1344
21110      909
20111      898
10100      763
10110      695
00110      418
00100      412
10101      292
21101      286
20100      256
20110      238
00101      149
20101       90
Name: stratum, dtype: int64

In [63]:
df_survey_twins['region'] = df_survey_twins['region'].replace({'Svealand':'0','Göteland':'1', 'Norrland':'2'})

df_survey_twins['urban_density'] = df_survey_twins['urban_density'].replace({'A': '0', 'B': '1', 'C': '1'})

df_survey_twins['avg_dist_cat'] = '0'
df_survey_twins['avg_dist_cat'][df_survey_twins.avg_dist > avg_dist_threshold] = '1'

df_survey_twins['com_dist_cat'] = '0'
df_survey_twins['com_dist_cat'][df_survey_twins.com_distance > com_dist_threshold] = '1'


In [64]:

df_survey_twins['stratum'] = df_survey_twins['region'].astype(str) + df_survey_twins['urban_density'].astype(str) +  df_survey_twins['commute'].astype(str)+  df_survey_twins['avg_dist_cat'].astype(str)

In [65]:

df_survey_twins['stratum'][(df_survey_twins.commute == 1)] = df_survey_twins['stratum'][(df_survey_twins.commute == 1)] + df_survey_twins['com_dist_cat'][(df_survey_twins.commute == 1)].astype(str)

df_survey_twins['stratum'].value_counts()

1100     2588
11100    1935
11111    1913
1101     1552
0100     1108
01100     853
01111     807
0101      614
1001      579
10111     546
11110     410
2100      241
21100     234
0001      174
01110     165
00111     155
11101     149
1000      125
2001      124
2101       95
10100      91
21111      90
20111      83
10110      82
01101      72
21110      29
00110      17
0000       14
00100      10
21101       9
10101       7
20110       3
20100       2
2000        1
Name: stratum, dtype: int64

In [66]:
list_survey_type = list(df_survey_twins['stratum'].unique())
list_MAD_type = list(df_MAD['stratum'].unique())


set(list_MAD_type).difference(set(list_survey_type))

{'00101', '20101'}

In [67]:
threshold_count = 50

df_stratum = pd.DataFrame(columns=['stratum','level','mad_size', 'sur_size','survey_ppl'])

In [68]:
#list_MAD_type= ['1100','1101']

for i in list_MAD_type:
    df_MAD_p_type = df_MAD[df_MAD['stratum']==i]
    #print('mad len: ', len(df_MAD_p_type))

    df_sur_p_type = df_survey_twins[df_survey_twins['region']==i[0]]

    df_sur_temp = df_sur_p_type
    level = 0
    if len(df_sur_p_type) > threshold_count:
        df_sur_temp = df_sur_p_type[df_sur_p_type['urban_density']==i[1]]
        level = 1
        if len(df_sur_temp) > threshold_count:
            df_sur_p_type = df_sur_temp
            df_sur_temp = df_sur_p_type[df_sur_p_type['commute']==int(i[2])]
            level = 2
            #print(level)
            if len(df_sur_temp) > threshold_count:
                df_sur_p_type = df_sur_temp
                df_sur_temp = df_sur_p_type[df_sur_p_type['avg_dist_cat']==i[3]]
                level = 3
                #print(int(i[2]))
                if (len(df_sur_temp) > threshold_count)&(int(i[2])==0):
                   df_sur_p_type = df_sur_temp
                   level = 4

                elif (len(df_sur_temp) > threshold_count)&(int(i[2])==1):
                    df_sur_p_type = df_sur_temp
                    df_sur_temp = df_sur_p_type[df_sur_p_type['com_dist_cat']==i[4]]
                    level = 4
                    #print(level)

                    if len(df_sur_temp) > threshold_count:
                        df_sur_p_type = df_sur_temp
                        level = 5

    df_stratum = df_stratum.append(dict(stratum=i, level=level, mad_size=len(df_MAD_p_type), sur_size=len(df_sur_p_type), survey_ppl=df_sur_p_type['sub_id'].tolist()), ignore_index=True)




In [69]:
# from collections import Counter
#
#
#
# def unique_activities(data):
#     act_tuple = data['act_seq']
#     #data['act_uniq'] = tuple(set(Counter(act_tuple).keys()))
#     data['act_uniq'] = tuple(Counter(act_tuple).keys())
#
#     # this will take only unique numbers from the tuple
#     return data
#
# tqdm.pandas()
# df_survey_twins = df_survey_twins.apply(lambda row: unique_activities(row), axis=1)

## simplify activity sequences

In [70]:

def act_seq_simplifier(data):
    alist = list(data['act_seq'])
    counter = 0
    for i in range(len(alist) - 1, 0, -1):
        if alist[i] in ['Home', 'Work', 'School']:
            counter = 0
            if alist[i] == alist[i-1]:
                del alist[i]
        elif alist[i] == 'Other':
            counter += 1
            if counter > 3:
                del alist[i]
    data['act_uniq'] = tuple(alist)
    return data

tqdm.pandas()
df_survey_twins = df_survey_twins.apply(lambda row: act_seq_simplifier(row), axis=1)

In [71]:
seq_dist = df_survey_twins.act_uniq.value_counts(normalize=True).reset_index(name='count_desired')
seq_dist['count_desired']=seq_dist['count_desired']*len(df_MAD)

## generate candidates df by stratums

In [72]:
df_candidates = pd.DataFrame(columns=['sub_id', 'act_seq', 'act_uniq','stratum','mad_size' ,'ratio'])


for index, row in df_stratum.iterrows():

    df_temp= df_survey_twins[['sub_id', 'act_seq', 'act_uniq']][df_survey_twins.sub_id.isin(row['survey_ppl'])]
    df_temp['stratum']= row['stratum']
    df_temp['mad_size']=row['mad_size']
    df_temp['ratio']=row['mad_size']/row['sur_size']
    df_candidates = df_candidates.append(df_temp)
    #print(row['survey_ppl'])
    #len(df_survey_twins[df_survey_twins.sub_id.isin(row['survey_ppl'])])



In [73]:
df_candidates.ratio.sum()

258666.99999999994

In [74]:
#df_candidates.groupby('act_uniq')['ratio'].sum()

## fitting probabilities to the simplified activity seq distribution

In [75]:
for i in range(40):

    #fit to candidates ratios to the act seq distribution
    bbb = df_candidates.groupby('act_uniq')['ratio'].sum().reset_index(name='count')
    merged_df = pd.merge(seq_dist, bbb, left_on='index', right_on='act_uniq', how='inner')
    merged_df['result_column'] = merged_df['count_desired'] / merged_df['count']
    df_candidates = pd.merge(df_candidates, merged_df[['act_uniq', 'result_column']], on='act_uniq')

    df_candidates['ratio']= df_candidates['ratio']*df_candidates['result_column']
    df_candidates.drop(columns='result_column', inplace=True)

    #fit to candidates ratios to the mad size by stratum
    bbb = df_candidates.groupby('stratum')['ratio'].sum().reset_index(name='count')
    df_candidates = pd.merge(df_candidates, bbb, on='stratum')

    df_candidates['ratio'] = df_candidates['ratio'] * (df_candidates['mad_size'] / df_candidates['count'])
    df_candidates.drop(columns='count', inplace=True)





In [76]:
#df_candidates.groupby('act_uniq')['ratio'].sum()


## sampling survey ppl by the calculated probabilities for each stratum

In [77]:
df_candidates['probability'] = df_candidates['ratio']/df_candidates['mad_size']

#df_MAD['twin_ppl'] =0

In [78]:
df_candidates_prob= df_candidates.groupby('stratum')['probability'].apply(list).reset_index(name='probability_list')


df_candidates_ppl= df_candidates.groupby('stratum')['sub_id'].apply(list).reset_index(name='ppl_list')

In [79]:
df_stratum = pd.merge(df_stratum, df_candidates_prob)

df_stratum = pd.merge(df_stratum, df_candidates_ppl)


In [80]:
df_stratum['twin_ppl'] = df_stratum.apply( lambda row: np.argmax(np.random.multinomial(1, row['probability_list'], row['mad_size']), axis=1).tolist()  , axis=1)

## matching sampled survey ppl with MAD ppl

In [81]:
df_survey_twins_new = df_survey_twins.set_index('sub_id')
df_survey_twins_new = df_survey_twins_new[['act_seq', 'act_uniq', 'avg_dist']]

In [82]:
from collections import Counter


def other_count(data):
    act_tuple = data['act_uniq']
    other_count_seq = Counter(act_tuple).get('Other')
    if other_count_seq is not None:
        data['other_count_seq'] = other_count_seq
    else:
        data['other_count_seq'] = 0
    return data

In [83]:

tqdm.pandas()
df_survey_twins_new = df_survey_twins_new.apply(lambda row: other_count(row), axis=1)



In [84]:


df_twins = pd.DataFrame()

for index, row in df_stratum.iterrows():
    twin_ppl_list = df_stratum.loc[index,'ppl_list']
    twin_ppl_place = df_stratum.loc[index,'twin_ppl']
    twin_ppl_list = [twin_ppl_list[place] for place in twin_ppl_place]

    df_survey_twins_new_candidates= df_survey_twins_new.loc[twin_ppl_list].reset_index()

    df_MAD_new = df_MAD[df_MAD.stratum==row['stratum']]

    zero_MAD = df_MAD_new[df_MAD_new.other_count == 0]
    zero_survey = df_survey_twins_new_candidates[df_survey_twins_new_candidates.other_count_seq == 0]

    #for i in range(len(df_MAD_new.other_count_seq.max())):
    if len(zero_MAD) <= len(zero_survey):

        zero_survey.sort_values(by=['avg_dist'], inplace=True)
        zero_survey.reset_index(inplace=True)
        zero_survey = zero_survey.iloc[:len(zero_MAD)]
        remain_survey = df_survey_twins_new_candidates.drop(index=zero_survey.index.to_list())
        zero_survey.drop(columns='index', inplace=True)

        zero_MAD.sort_values(by=['avg_dist'], inplace=True)
        zero_MAD.reset_index(inplace=True)
        zero_MAD.drop(columns='index', inplace=True)

        zero_MAD = pd.concat([zero_MAD, zero_survey.iloc[:len(zero_MAD)]], axis=1, ignore_index=False)


    else:
        print(row['stratum'])

    df_twins = df_twins.append(zero_MAD)

    remain_MAD = df_MAD_new[df_MAD_new.other_count > 0]


    remain_survey.sort_values(by=['avg_dist'], inplace=True)
    remain_survey.reset_index(inplace=True)
    remain_survey.drop(columns='index', inplace=True)

    remain_MAD.sort_values(by=['avg_dist'], inplace=True)
    remain_MAD.reset_index(inplace=True)
    remain_MAD.drop(columns='index', inplace=True)

    remain_MAD = pd.concat([remain_MAD, remain_survey], axis=1, ignore_index=False)
    df_twins = df_twins.append(remain_MAD)

    #print(index)





In [85]:


# df_twins = pd.DataFrame()
#
# for index, row in df_stratum.iterrows():
#     twin_ppl_list = df_stratum.loc[index,'ppl_list']
#     twin_ppl_place = df_stratum.loc[index,'twin_ppl']
#     twin_ppl_list = [twin_ppl_list[place] for place in twin_ppl_place]
#
#     df_survey_twins_new_candidates= df_survey_twins_new.loc[twin_ppl_list].reset_index()
#
#     df_survey_twins_new_candidates.sort_values(by=['avg_dist'], inplace=True)
#     df_survey_twins_new_candidates.reset_index(inplace=True)
#     df_survey_twins_new_candidates.drop(columns='index', inplace=True)
#
#     df_MAD_new = df_MAD[df_MAD.stratum==row['stratum']]
#     df_MAD_new.sort_values(by=['avg_dist'], inplace=True)
#     df_MAD_new.reset_index(inplace=True)
#     df_MAD_new.drop(columns='index', inplace=True)
#
#     df_MAD_new = pd.concat([df_MAD_new, df_survey_twins_new_candidates], axis=1, ignore_index=False)
#     df_twins = df_twins.append(df_MAD_new)
#
#     #print(index)
#
#



In [86]:
#kkk = df_twins[['uid','other_count','other_count_seq']]


#kkk = df_twins[df_twins['other_count']==0]

## Evaluation

In [87]:
print('The share of the matched survey ppl: ', np.round(len(df_twins.sub_id.unique())/len(df_survey_twins.sub_id.unique()),2))

The share of the matched survey ppl:  1.0


In [88]:
df_twins.sub_id.value_counts()

20110841052    586
20120941012    123
20112431063    116
20113551018    111
20142021076    109
              ... 
20110621039      1
20124811025      1
20121241027      1
20120651018      1
20122021027      1
Name: sub_id, Length: 14818, dtype: int64

In [89]:
df_survey_twins.act_uniq.value_counts(normalize=True)


(Home, Other, Home)                                                                                          0.220206
(Home, Work, Home)                                                                                           0.217383
(Home, Other, Home, Other, Home)                                                                             0.059219
(Home, Other, Other, Home)                                                                                   0.057740
(Home, Work, Other, Home)                                                                                    0.051422
                                                                                                               ...   
(Work, Home, Other, Home, Other, Other, Home)                                                                0.000067
(Home, Work, Other, Other, Other, Work, Other, Other, Work, Other, Work, Other, Other, Work, Other, Home)    0.000067
(Home, Other, Other, Home, Other, Other, School, Home)  

In [90]:
aac= df_twins.groupby('act_uniq')['wt'].sum()/df_twins['wt'].sum()
aac= aac.reset_index(name='share').sort_values(by='share', ascending=False)
aac

,act_uniq,share
322,"(Home, Work, Home)",0.242685
0,"(Home, Other, Home)",0.200858
1,"(Home, Other, Home, Other, Home)",0.055424
367,"(Home, Work, Other, Home)",0.053444
323,"(Home, Work, Home, Other, Home)",0.049709
...,...,...
107,"(Home, Other, Other, Home, School, Home, Other...",0.000013
502,"(Work, Other, Work, Other, Home, Other, Home, ...",0.000012
105,"(Home, Other, Other, Home, Other, Work, Other,...",0.000006
278,"(Home, Other, Work, Other, Work, Other, Work)",0.000006


In [91]:
df_survey_twins.act_seq.value_counts(normalize=True)


(Home, Work, Home)                                                              0.192982
(Home, Other, Home)                                                             0.186933
(Home, Other, Home, Other, Home)                                                0.051691
(Home, Other, Other, Home)                                                      0.050212
(Home, Work, Other, Home)                                                       0.045910
                                                                                  ...   
(Home, Other, Home, Other, Work, Home, Other, Home)                             0.000067
(Home, Other, School, Home, Work, Other, Home)                                  0.000067
(Home, Other, Home, Work, Other, Work, Other, Home)                             0.000067
(Home, Other, Other, School, School, Home)                                      0.000067
(Home, Other, Home, Home, Work, Home, Other, Home, Other, Home, Other, Home)    0.000067
Name: act_seq, Length

In [92]:
aac= df_twins.groupby('act_seq')['wt'].sum()/df_twins['wt'].sum()
aac.reset_index(name='share').sort_values(by='share', ascending=False)

,act_seq,share
624,"(Home, Work, Home)",2.135517e-01
133,"(Home, Other, Home)",1.718277e-01
152,"(Home, Other, Home, Other, Home)",4.859770e-02
691,"(Home, Work, Other, Home)",4.764537e-02
636,"(Home, Work, Home, Other, Home)",4.681306e-02
...,...,...
322,"(Home, Other, Other, Other, Home, Other, Other...",3.300587e-06
323,"(Home, Other, Other, Other, Home, Other, Other...",1.348395e-06
40,"(Home, Home, Other, Home, Other, Home, Home, O...",1.153196e-06
144,"(Home, Other, Home, Home, Other, Home, Other, ...",5.661905e-07


# code ends here